In [ ]:
! pip install --upgrade codetiming numpy pandas census us mechanicalsoup geopandas pandas_bokeh torch
get_ipython().kernel.do_shutdown(True)

In [ ]:
mount_path = '/content/drive'
repo_path = f'{mount_path}/MyDrive/gerrymandering/2022-10/voting_predictor'
from google.colab import drive
drive.mount(mount_path)
%cd {repo_path}
%load_ext google.colab.data_table
%load_ext autoreload
%autoreload
from model import *
%cd {repo_path}
%load_ext google.colab.data_table


In [ ]:
%load_ext autoreload
%autoreload
from model import *

a = ['s1501_c01_003ea']
get(a, 'acs5st', 2018, 'tract')
# census_session.acs5.state(('NAME', 'S1501_C01_053E'), '01')

In [ ]:
5284-85-5144, 3467-85-2688

In [ ]:
%load_ext autoreload
%autoreload
from model import *

class Feedforward(torch.nn.Module):
    def __init__(self, input_size, hidden_size, activation):
        super(Feedforward, self).__init__()
        assert len(hidden_size) == len(activation), 'hidden_size and activation must have same length'
        L = []
        p = input_size
        for q, f in zip(hidden_size, activation):
            L.append(torch.nn.Linear(p, q))
            L.append(f())
            p = q
        self.nn = torch.nn.Sequential(*L)
        self.rmse_train = []
        self.rmse_test  = []

    def forward(self, X, W):
        self.prop = self.nn(X)
        self.pred = ((self.prop * W) @ B).squeeze()
        return self.pred


def features(year, election='general', office='President', level='tract', overwrite=False):
    get_acs5(year=year, overwrite=overwrite)
    pq = data_path / f'features/{year}_{election}_{office}.parquet'
    if pq.is_file() and not overwrite:
        df = pd.read_parquet(pq)
    else:
        qry = f"""
        select
            vtd2020,
            election,
            office,
            lower(party) as party,
            campaign,
            -- party,
            votes,
        from
            elections.all
        where
            election = '{election}'
            and office = '{office}'
            and year = {year}
            and party in ('R', 'D')
        """
        elections = query_to_df(qry).pivot(index=['vtd2020', 'campaign'], columns='party', values='votes').fillna(0)#.reset_index('campaign')#.set_index('vtd2020')

        qry = f"""
        select
            A.vtd2020,
            A.county,
            cast(round(A.aland)             as int) as aland,
            cast(round(A.awater)            as int) as awater,
            cast(round(A.atot)              as int) as atot,
            cast(round(A.perim)             as int) as perim,
            cast(round(A.polsby_popper*100) as int) as polsby_popper,
            cast(round(A.dist_to_border)    as int) as dist_to_border,
        from
            shapes.vtd2020 as A
        """
        shapes = query_to_df(qry).set_index('vtd2020')

        qry = f"""select
            *
        from
            acs5.{year}
        """
        acs = query_to_df(qry).set_index('vtd2020')

        vtd = prep(elections.join(shapes, how='outer').join(acs, how='outer'))
        df = vtd

        for race in ['all', 'white', 'hisp', 'other']:
            df[race+'_vap_density'] = (df[race+'_vap_pop'] / np.fmax(df['aland'], 1) * 1609.34**2)
        df['votes'] = df[elections.columns].sum(axis=1)
        for party in elections.columns:
            df[party+'_pct'] = df[party] / np.fmax(df['all_tot_pop'], 1) * 100

        # for col in df.columns:
        #     a = col.find('_')+1
        #     b = col[a:].find('_')+a+1
        #     subpop = col[:b] + 'pop'
        #     if col[b:] != 'pop' and subpop in df.columns:
        #         df[col] /= np.fmax(df[subpop], 1)
        df_to_parquet(df, pq)
    
    return pq


elections = ['2020_general_President', '2016_general_President', '2018_general_USSen', '2020_general_USSen']
feat = [
    'polsby_popper', 'dist_to_border',
    'hisp_vap_spanish_at_home', 'hisp_vap_spanish_at_home_english_well',
    'all_tot_pop'       , 'white_tot_pop'       , 'hisp_tot_pop'       , 'other_tot_pop',
    'all_vap_pop'       , 'white_vap_pop'       , 'hisp_vap_pop'       , 'other_vap_pop',
    'all_vap_density'   , 'white_vap_density'   , 'hisp_vap_density'   , 'other_vap_density',
    'all_vap_poverty'   , 'white_vap_poverty'   , 'hisp_vap_poverty'   , 'other_vap_poverty',
    'all_vap_elderly'   , 'white_vap_elderly'   , 'hisp_vap_elderly'   , 'other_vap_elderly',
    'all_vap_highschool', 'white_vap_highschool', 'hisp_vap_highschool', 'other_vap_highschool',
]
targ = ['d', 'r']

pqs = {elec: features(*elec.split('_')) for elec in elections}
for elec, pq in pqs.items():
    print(elec)

    dfs = {elec: parquet_to_df(pq)[feat+targ].sample(frac=1).astype(float) for elec, pq in pqs.items()}
    display(dfs[elec].head(2))
    
    break
    # test = dfs.pop(elec)
    # train = pd.concat(dfs.values())
    # X_test  = torch.FloatTensor(test [targ].values).to(device)
    # Y_test  = torch.FloatTensor(test [feat].values).to(device)
    # X_train = torch.FloatTensor(train[targ].values).to(device)
    # Y_train = torch.FloatTensor(train[feat].values).to(device)

    # model = Feedforward(
    #     input_size = X_train.shape[1],
    #     hidden_size = (50, Y_train.shape[1]),
    #     activation = (torch.nn.ReLU, torch.nn.Sigmoid)
    # ).to(device)
    # loss_fcn = torch.nn.MSELoss()
    # optimizer = torch.optim.Adam(model.parameters())
    
    # steps = 2
    # for k in range(steps):
    #     optimizer.zero_grad()  # Clear gradient
    #     loss = loss_fcn(model(X_train), Y_train) # Compute train loss
    #     loss.backward()  # Backward propagation
    #     optimizer.step()  # Learn
    # break



In [ ]:
train.describe()

In [ ]:
class Feedforward(torch.nn.Module):
    def __init__(self, input_size, hidden_size, activation):
        super(Feedforward, self).__init__()
        activation = listify(activation)
        hidden_size = listify(hidden_size)
        q = hidden_size[0]
        L = [torch.nn.Linear(input_size, q)]
        p = q
        for f, q in zip(activation, hidden_size):
            L.append(f())
            L.append(torch.nn.Linear(p, q))
            p = q
        self.nn = torch.nn.Sequential(*L)

    def forward(self, x, w):
        self.prop = self.model(x)
        self.pred = ((self.prop * w) @ b).squeeze()
        return self.pred




model = Feedforward(
    input_size = x_train.shape[1],
    hidden_size = (50, 6),
    activation = (torch.nn.ReLU, torch.nn.Sigmoid)
).to(device)
loss_fcn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

model.eval()
print(f'Test loss before training {loss_fcn(model(x_test, w_test), y_test)}')

model.train()
epoch = 1000
for epoch in range(epoch):
    optimizer.zero_grad()             # Clear gradient
    y_pred = model(x_train, w_train)  # Forward propagation
    loss = loss_fcn(y_pred, y_train)  # Compute Loss
    if epoch % 100 == 0:
        print(f'Epoch {epoch}: train loss {loss}')
    loss.backward()                   # Backward propagation
    optimizer.step()                  # Learn

model.eval()
print(f'Test loss after training {loss_fcn(model(x_test, w_test), y_test)}')